In [2]:
from typing import List
from pathlib import Path

def read_text(path: str) -> str:
    for encoding in ("utf-8-sig", "utf-8", "cp932"):
        try:
            return Path(path).read_text(encoding=encoding)
        except UnicodeDecodeError:
            pass
    return Path(path).read_text(encoding="utf-8", errors="replace")

def split_into_chunks(doc_file: str) -> List[str]:
    content = read_text(doc_file)

    return [chunk for chunk in content.split("\n\n")]

chunks = split_into_chunks("doc_test2.md")

for i, chunk in enumerate(chunks):
    print(f"[{i}] {chunk}\n")

[0] # ドラえもんと超サイヤ人：時空の戦い

[1] あるいつもの昼下がり。のび太は相変わらず机の前でぼんやりし、宿題は山のように積まれているのに、まだ1ページ目すら手を付けていなかった。ドラえもんは横で漫画をめくりながら、ときおりため息をつく。「この子は本当に、いつまでたっても頼りないなあ」と思っていたその時——。

[2] 突然、空からまばゆい強光が降り注ぎ、部屋全体が激しく揺れた。光の中から現れたのは、金髪の少年。戦闘服をまとい、圧倒的な気迫を放っている。未来から来た超サイヤ人——トランクスだった。彼は現れるなり、衝撃的な言葉を告げる。
「未来の地球は、まもなく闇の勢力によって滅ぼされる。助けが必要だ。ドラえもん、君の力を貸してほしい。」

[3] ドラえもんとのび太は驚きつつも、トランクスの揺るぎない眼差しから、拒めない覚悟を感じ取った。トランクスは続けて説明する。未来の敵は、ただの悪役ではない。「闇のサイヤ人」と呼ばれる存在で、邪悪な科学者がベジータの遺伝子を複製し、さらに改造を施して生み出した怪物だという。戦闘力は桁外れで、しかも歪んだ時間エネルギーを操り、ほとんど誰も太刀打ちできない。トランクスは長年ひとりで戦ってきたが、そのたびに惨敗してきた。彼は言った。「僕の時代に足りなかった唯一の武器は“科学”だ。でも君たちは、それを持っている。」

[4] こうしてドラえもんは、トランクスとのび太を連れ、タイムマシンを起動して、崩壊寸前の未来世界へ向かう。到着した先の光景は凄惨だった。都市は瓦礫の山と化し、大地には無数の亀裂が走り、空には息苦しいほど重い黒い霧が漂っている。トランクスは言う。「これが闇のサイヤ人のせいだ。ほとんどの命が消され、残っているのは僕が必死に踏みとどまっているだけなんだ。」

[5] のび太は恐怖で足がすくみそうになる。しかし、無関係な人々が理不尽に傷つく姿を目の当たりにし、胸の奥に小さな闘志が灯る。ドラえもんは冷静に状況を分析し、闇の勢力に対抗するため、最強クラスの秘密道具を三つ使う決断をした。

[6] 三つの秘密道具はこうだ。一時的に超戦闘力を付与できる「コピー・マント」、時間を5秒だけ止められる「時間停止ウォッチ」、1分の間に1年分の修行ができる「精神と時の部屋・携帯版」。のび太は携帯版の精神と時の部屋に押し込まれ、超高密度

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("sentence-transformers/LaBSE")
#embedding_model = SentenceTransformer("sonoisa/sentence-bert-base-ja-mean-tokens-v2")


def embed_chunk(chunk: str) -> List[float]:
    embedding = embedding_model.encode(chunk, normalize_embeddings=True)
    return embedding.tolist()


embedding = embed_chunk("テストです。")
print(len(embedding))
print(embedding)

In [4]:
embeddings = [embed_chunk(chunk) for chunk in chunks]

print(len(embeddings))
print(embeddings[0])

15
[-0.06110594794154167, -0.035542070865631104, -0.008514412678778172, 0.008080102503299713, 0.004735893569886684, -0.03752667084336281, 0.038748063147068024, 0.05020720511674881, -0.004357959609478712, -0.01833692193031311, 0.04916316270828247, 0.03610991686582565, -0.07141713052988052, -0.047785788774490356, 0.001504462561570108, 0.009472737088799477, -0.029980378225445747, -0.042525216937065125, -0.010845967568457127, 0.012759956531226635, 0.016269372776150703, 0.00853117648512125, -0.03166016936302185, -0.06714751571416855, -0.09291373193264008, -0.08767123520374298, 0.06426317989826202, 0.05537636950612068, -0.038745008409023285, 0.0013697294052690268, -0.03067336417734623, -0.017645923420786858, -0.004084862302988768, -0.02161289192736149, -0.0041930945590138435, 0.00032631726935505867, -0.0314619354903698, 0.04863067716360092, 0.054334450513124466, -0.013091684319078922, -0.021450521424412727, 0.011503899469971657, 0.012988653033971786, -0.04282352700829506, -0.0183946453034877

In [5]:
import chromadb

chromadb_client = chromadb.EphemeralClient()
chromadb_collection = chromadb_client.get_or_create_collection(name="default")

def save_embeddings(chunks: List[str], embeddings: List[List[float]]) -> None:
    for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
        chromadb_collection.add(
            documents=[chunk],
            embeddings=[embedding],
            ids=[str(i)]
        )

save_embeddings(chunks, embeddings)

In [6]:
def retrieve(query: str, top_k: int) -> List[str]:
    query_embedding = embed_chunk(query)
    results = chromadb_collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    return results['documents'][0]

#query = "哆啦A梦使用的3个秘密道具分别是什么？"
#query = "大雄使用了哪些道具，分别是什么？"
query = "のび太が使った道具は何ですか？役割は何ですか？"

retrieved_chunks = retrieve(query, 7)

for i, chunk in enumerate(retrieved_chunks):
    print(f"[{i}] {chunk}\n")

[0] ドラえもんとのび太は驚きつつも、トランクスの揺るぎない眼差しから、拒めない覚悟を感じ取った。トランクスは続けて説明する。未来の敵は、ただの悪役ではない。「闇のサイヤ人」と呼ばれる存在で、邪悪な科学者がベジータの遺伝子を複製し、さらに改造を施して生み出した怪物だという。戦闘力は桁外れで、しかも歪んだ時間エネルギーを操り、ほとんど誰も太刀打ちできない。トランクスは長年ひとりで戦ってきたが、そのたびに惨敗してきた。彼は言った。「僕の時代に足りなかった唯一の武器は“科学”だ。でも君たちは、それを持っている。」

[1] そして決戦は、闇のサイヤ人の空中要塞の前で始まった。トランクスが先陣を切り、全力で真正面からぶつかる。ドラえもんはどこでもドアや秘密道具で支援し、あらゆる方向から攪乱して、敵の時空操作をできるだけ封じようとする。だが闇のサイヤ人はあまりに強大で、トランクスひとりでは押し切れない。まして倒すなど到底無理に思えた。トランクスがついに追い詰められ、倒されかけたその瞬間——。コピー・マントをまとったのび太が、恐怖を突き破り、上空から飛び込んできた。拳は金色の炎のような光をまとい、その狙いは敵の心臓部へ一直線。

[2] 現代へ戻ったのび太は、まるで別人のようだった。すぐに文句を言わず、責任から逃げない。宿題をきちんと終え、母の買い物を手伝い、さらには自分から運動の練習まで始めた。ドラえもんは驚いて言葉を失う。これは気まぐれではない。のび太の心の奥で、本当の意味で何かが変わったのだ。

[3] 三つの秘密道具はこうだ。一時的に超戦闘力を付与できる「コピー・マント」、時間を5秒だけ止められる「時間停止ウォッチ」、1分の間に1年分の修行ができる「精神と時の部屋・携帯版」。のび太は携帯版の精神と時の部屋に押し込まれ、超高密度の訓練を受ける。現実では数分しか経っていないのに、彼は“丸一年”の苦しい修行を経験することになった。最初はいつもの弱さが顔を出し、「やめたい」「逃げたい」と思ってしまう。だが、しずかちゃんの笑顔、両親の姿、そしてドラえもんの揺るがない眼差しを思い出し、のび太は歯を食いしばって踏みとどまった。部屋から出てきたのび太は、心身ともに見違えるほど変わっていた。目には怯えではなく、成熟した自信が宿っている。

[4] あるいつもの昼下がり。のび太は相変わら

In [8]:
from sentence_transformers import CrossEncoder

def rerank(query: str, retrieved_chunks: List[str], top_k: int) -> List[str]:
    cross_encoder = CrossEncoder('cross-encoder/mmarco-mMiniLMv2-L12-H384-v1')
    pairs = [(query, chunk) for chunk in retrieved_chunks]
    scores = cross_encoder.predict(pairs)

    scored_chunks = list(zip(retrieved_chunks, scores))
    scored_chunks.sort(key=lambda x: x[1], reverse=True)

    return [chunk for chunk, _ in scored_chunks][:top_k]

reranked_chunks = rerank(query, retrieved_chunks, 5)

for i, chunk in enumerate(reranked_chunks):
    print(f"[{i}] {chunk}\n")

[0] 三つの秘密道具はこうだ。一時的に超戦闘力を付与できる「コピー・マント」、時間を5秒だけ止められる「時間停止ウォッチ」、1分の間に1年分の修行ができる「精神と時の部屋・携帯版」。のび太は携帯版の精神と時の部屋に押し込まれ、超高密度の訓練を受ける。現実では数分しか経っていないのに、彼は“丸一年”の苦しい修行を経験することになった。最初はいつもの弱さが顔を出し、「やめたい」「逃げたい」と思ってしまう。だが、しずかちゃんの笑顔、両親の姿、そしてドラえもんの揺るがない眼差しを思い出し、のび太は歯を食いしばって踏みとどまった。部屋から出てきたのび太は、心身ともに見違えるほど変わっていた。目には怯えではなく、成熟した自信が宿っている。

[1] のび太は恐怖で足がすくみそうになる。しかし、無関係な人々が理不尽に傷つく姿を目の当たりにし、胸の奥に小さな闘志が灯る。ドラえもんは冷静に状況を分析し、闇の勢力に対抗するため、最強クラスの秘密道具を三つ使う決断をした。

[2] そして決戦は、闇のサイヤ人の空中要塞の前で始まった。トランクスが先陣を切り、全力で真正面からぶつかる。ドラえもんはどこでもドアや秘密道具で支援し、あらゆる方向から攪乱して、敵の時空操作をできるだけ封じようとする。だが闇のサイヤ人はあまりに強大で、トランクスひとりでは押し切れない。まして倒すなど到底無理に思えた。トランクスがついに追い詰められ、倒されかけたその瞬間——。コピー・マントをまとったのび太が、恐怖を突き破り、上空から飛び込んできた。拳は金色の炎のような光をまとい、その狙いは敵の心臓部へ一直線。

[3] ドラえもんとのび太は驚きつつも、トランクスの揺るぎない眼差しから、拒めない覚悟を感じ取った。トランクスは続けて説明する。未来の敵は、ただの悪役ではない。「闇のサイヤ人」と呼ばれる存在で、邪悪な科学者がベジータの遺伝子を複製し、さらに改造を施して生み出した怪物だという。戦闘力は桁外れで、しかも歪んだ時間エネルギーを操り、ほとんど誰も太刀打ちできない。トランクスは長年ひとりで戦ってきたが、そのたびに惨敗してきた。彼は言った。「僕の時代に足りなかった唯一の武器は“科学”だ。でも君たちは、それを持っている。」

[4] あるいつもの昼下がり。のび太は相変わらず机の前でぼんやりし、宿題は山のように積まれている

In [9]:
from dotenv import load_dotenv, find_dotenv
from google import genai

import os

dotenv_path = find_dotenv(usecwd=True)
load_dotenv(dotenv_path, override=True)
api_key = os.getenv("GEMINI_API_KEY")
#print("GEMINI_API_KEY loaded:", bool(api_key))
google_client = genai.Client(api_key=api_key) if api_key else genai.Client()

def generate(query: str, chunks: List[str]) -> str:
    prompt = f"""あなたは知識アシスタントです。ユーザーの質問と、以下の断片（チャンク）に基づいて、正確な回答を生成してください。日本語に翻訳。

問題: {query}

チャンク:
{"\n\n".join(chunks)}

上記の内容に基づいて回答し、記載のない情報は作りません。日本語で対応します。"""

    print(f"{prompt}\n\n---\n")

    response = google_client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt
    )

    return response.text

answer = generate(query, reranked_chunks)
print(answer)

あなたは知識アシスタントです。ユーザーの質問と、以下の断片（チャンク）に基づいて、正確な回答を生成してください。日本語に翻訳。

問題: のび太が使った道具は何ですか？役割は何ですか？

チャンク:
三つの秘密道具はこうだ。一時的に超戦闘力を付与できる「コピー・マント」、時間を5秒だけ止められる「時間停止ウォッチ」、1分の間に1年分の修行ができる「精神と時の部屋・携帯版」。のび太は携帯版の精神と時の部屋に押し込まれ、超高密度の訓練を受ける。現実では数分しか経っていないのに、彼は“丸一年”の苦しい修行を経験することになった。最初はいつもの弱さが顔を出し、「やめたい」「逃げたい」と思ってしまう。だが、しずかちゃんの笑顔、両親の姿、そしてドラえもんの揺るがない眼差しを思い出し、のび太は歯を食いしばって踏みとどまった。部屋から出てきたのび太は、心身ともに見違えるほど変わっていた。目には怯えではなく、成熟した自信が宿っている。

のび太は恐怖で足がすくみそうになる。しかし、無関係な人々が理不尽に傷つく姿を目の当たりにし、胸の奥に小さな闘志が灯る。ドラえもんは冷静に状況を分析し、闇の勢力に対抗するため、最強クラスの秘密道具を三つ使う決断をした。

そして決戦は、闇のサイヤ人の空中要塞の前で始まった。トランクスが先陣を切り、全力で真正面からぶつかる。ドラえもんはどこでもドアや秘密道具で支援し、あらゆる方向から攪乱して、敵の時空操作をできるだけ封じようとする。だが闇のサイヤ人はあまりに強大で、トランクスひとりでは押し切れない。まして倒すなど到底無理に思えた。トランクスがついに追い詰められ、倒されかけたその瞬間——。コピー・マントをまとったのび太が、恐怖を突き破り、上空から飛び込んできた。拳は金色の炎のような光をまとい、その狙いは敵の心臓部へ一直線。

ドラえもんとのび太は驚きつつも、トランクスの揺るぎない眼差しから、拒めない覚悟を感じ取った。トランクスは続けて説明する。未来の敵は、ただの悪役ではない。「闇のサイヤ人」と呼ばれる存在で、邪悪な科学者がベジータの遺伝子を複製し、さらに改造を施して生み出した怪物だという。戦闘力は桁外れで、しかも歪んだ時間エネルギーを操り、ほとんど誰も太刀打ちできない。トランクスは長年ひとりで戦ってきたが、そのたびに惨敗してきた。彼は言った。「僕の時代に足